<a href="https://colab.research.google.com/github/Haaretzz/Teste---Reclame-aqui/blob/master/Teste_Reclame_Aqui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

#Eu fiz upload do arquivo no meu github, para que não necessite fazer upload aqui no colab.
#Desse jeito eu posso mandar o link pelo questionário para que vocês consigam ver normalmente ;)
path = 'https://raw.githubusercontent.com/Haaretzz/Teste---Reclame-aqui/master/base_teste.csv'

#Inicialmente eu acabei tendo um problema com a separação automática do arquivo, já que
#há virgulas nos títulos, fazendo com que o python visualizasse mais de uma coluna na frase.
#O outro ponto foi o encoding, este erro provém possívelmente por conta de acentuação mesmo, assim
#acabei tendo que mudar para o "ISO-8859-1, este que acaba lidando melhor com os nossos acentuos."
base = pd.read_csv(path, sep = ';', encoding = 'ISO-8859-1')

#Só para ver o retorno:
base.head()

,id,titulo,publicacao
0,44121,Farfetch não estornou valor da devolução,2019-11-27 13:34:08 UTC
1,98294,Péssimo por venda,2019-01-16 16:41:44 UTC
2,2010,falta de respeito com as revendedoras,2019-11-05 22:41:49 UTC
3,32061,Desistência e cancelamento do Programa,2019-01-28 11:04:27 UTC
4,270,Serviço prioritário,2020-01-30 08:16:32 UTC


#Pergunta 1: Quantas reclamações tem, respectivamente, as empresas com os seguintes IDs: 7712 e mvrtggDcqcALclzf.

In [2]:
#Muito simples, apenas filtrei e agrupei pelos ID's:
base[['id', 'titulo']].query("id in ['7712', 'mvrtggDcqcALclzf']").groupby('id').count()

,titulo
id,
7712,808
mvrtggDcqcALclzf,448


#Pergunta 2: Quantas reclamações foram publicadas no da 18/09/2019?

In [3]:
#Primeira coisa que farei é passar os dados da base inicial para uma nova
#(sempre faço isso quando vou modificar colunas, assim não corro o risco de avariar a base)
base_com_data_separada = base.copy()

#Agora eu farei um split na coluna de publicação, assim separaremos a data e hora:
#O split é um método muito comúm quando falamos de string, ele é utilizado para
#separar uma string em várias outras a partir de um caractere passado como referência.
#E neste caso eu passei o ' ', que é basicamente um espaço, além disso, utilizei o "expand = True"
#para que fosse retornado um DataFrame do pandas ao invés de uma series, assim, podendo
#colocar diretamente na minha base de dados.
base_com_data_separada[['data', 'hora', 'formato']] = base['publicacao'].str.split(' ', expand = True)

#Visualizando como ficou:
base_com_data_separada.head()

,id,titulo,publicacao,data,hora,formato
0,44121,Farfetch não estornou valor da devolução,2019-11-27 13:34:08 UTC,2019-11-27,13:34:08,UTC
1,98294,Péssimo por venda,2019-01-16 16:41:44 UTC,2019-01-16,16:41:44,UTC
2,2010,falta de respeito com as revendedoras,2019-11-05 22:41:49 UTC,2019-11-05,22:41:49,UTC
3,32061,Desistência e cancelamento do Programa,2019-01-28 11:04:27 UTC,2019-01-28,11:04:27,UTC
4,270,Serviço prioritário,2020-01-30 08:16:32 UTC,2020-01-30,08:16:32,UTC


In [4]:
#Agora é só filtrar pela data:
base_com_data_separada[['id', 'data']].query("data == '2019-09-18'")['data'].count()

650

#Pergunta 3: Quantas reclamações foram publicadas entre as 13 e 14h do dia 22/11/2019?

In [5]:
#Primeiro vou filtrar toda a base de dados pela data solicitada e extrair apenas as colunas "data e hora":
base_data_filtrada = base_com_data_separada[['data','hora']].query("data == '2019-11-22'")
#Depois disso é só fazer uma query para achar os valores que estão entre as horas pedidas:
base_data_filtrada.query("hora >= '13:00:00' & hora <= '14:00:00'").count()['hora']

48

#Pergunta 4: Respectivamente, quantas reclamações têm, no título, os termos cobrança e cobrança indevida em toda a base?

##Ocorrências das palavras com o "Ç"

In [6]:
#Primeiro vamos criar uma nova base para não ficar confuso:
base_auxiliar = base.copy()

#Agora vou colocar todos os títulos em maíusculo, assim não terei problemas com "Case Sensitivity".
#Para isso eu pego apenas a coluna 'titulo', o que torna meu DataFrame agora uma series e utilizo o
#"upper()" fazendo com que os caracteres fiquem todos em maiúsculo:
base_auxiliar['titulo'] = base_auxiliar['titulo'].str.upper()

#Para ver como ficou:
base_auxiliar.head()

,id,titulo,publicacao
0,44121,FARFETCH NÃO ESTORNOU VALOR DA DEVOLUÇÃO,2019-11-27 13:34:08 UTC
1,98294,PÉSSIMO POR VENDA,2019-01-16 16:41:44 UTC
2,2010,FALTA DE RESPEITO COM AS REVENDEDORAS,2019-11-05 22:41:49 UTC
3,32061,DESISTÊNCIA E CANCELAMENTO DO PROGRAMA,2019-01-28 11:04:27 UTC
4,270,SERVIÇO PRIORITÁRIO,2020-01-30 08:16:32 UTC


In [7]:
#Agora irei criar uma coluna nova, para que possa dividir entre "cobrança" e "cobrança indevida":
#Para fazer isso, eu peguei apenas uma coluna do nosso DataFrame, fazendo com que ele vire uma series, 
#assim, eu pude transformá-lo em string e utilizar o "contains()", esta função retorna
#o index da linha e dados booleanos (False, True), então utilizando o "astype(int)"
#transformei os dados booleanos em 0 e 1, passando-os para uma nova coluna. Depois disso
#acaba ficando bem fácil de distinguir as ocorrências.
#OBS: Tive que utilizar o contains com a "COBRANÇA INDEVIDA|COBRANÇAS INDEVIDAS", pois se 
#colocasse "COBRANÇA" acabaria não conseguindo diferenciar, afinal, tudo iria vir com "1".
base_auxiliar['tipo_de_cobranca'] = base_auxiliar['titulo'].str.contains("COBRANÇA INDEVIDA|COBRANÇAS INDEVIDAS",
                                                                         na=False).astype(int)

#Agora eu preciso filtrar novamente tudo, porém pela ocorrência apenas da "COBRANÇA".
#Fazendo isso, irei passar o df para outra variável por questões de organização
#e clareza do código:                                                                         
base_cobranca = base_auxiliar[base_auxiliar['titulo'].str.contains('COBRANÇA')].copy()

#Agora é só agrupar e contar os valores:
base_cobranca.groupby('tipo_de_cobranca').count()['id']
#Sendo o 1 as "cobranças indevidas" e o 0 apenas a ocorrência da palavra "cobrança"

tipo_de_cobranca
0    5516
1    7658
Name: id, dtype: int64

In [8]:
#O total é:
base_cobranca.count()['id']

13174

##Ocorrências das palavras sem o "Ç"

In [9]:
#Agora para completar eu pegarei também a ocorrência da palavra cobrança sem o "Ç".
#Para isto, irei refazer todo o processo anterior:

base_auxiliar_2 = base.copy()

base_auxiliar_2['titulo'] = base_auxiliar_2['titulo'].str.upper()

base_auxiliar_2.head()

,id,titulo,publicacao
0,44121,FARFETCH NÃO ESTORNOU VALOR DA DEVOLUÇÃO,2019-11-27 13:34:08 UTC
1,98294,PÉSSIMO POR VENDA,2019-01-16 16:41:44 UTC
2,2010,FALTA DE RESPEITO COM AS REVENDEDORAS,2019-11-05 22:41:49 UTC
3,32061,DESISTÊNCIA E CANCELAMENTO DO PROGRAMA,2019-01-28 11:04:27 UTC
4,270,SERVIÇO PRIORITÁRIO,2020-01-30 08:16:32 UTC


In [10]:
base_auxiliar_2['tipo_de_cobranca'] = base_auxiliar_2['titulo'].str.contains('COBRANCA INDEVIDA|COBRANCAS INDEVIDAS',
                                                                             na=False).astype(int)

base_cobranca_2 = base_auxiliar_2[base_auxiliar_2['titulo'].str.contains('COBRANCA')].copy()

base_cobranca_2.groupby('tipo_de_cobranca').count()['id']

tipo_de_cobranca
0    151
1    182
Name: id, dtype: int64

In [11]:
#O total é:
base_cobranca_2.count()['id']

333